In [1]:
%load_ext lab_black

In [2]:
import requests
import urllib.request
import numpy as np

In [3]:
import pandas as pd
import gspread
from oauth2client.service_account import ServiceAccountCredentials

In [4]:
import altair as alt
import altair_grid as altgrid

alt.themes.register("grid", altgrid.theme)
alt.themes.enable("grid")

ThemeRegistry.enable('grid')

In [5]:
from datawrapper import Datawrapper

dw = Datawrapper(
    access_token="FtIwtvFtoGLaRT9a3gjX69PLu4wSuRyKddoOz6SOPw3k9wWyNICMHTkcPhOGCR5Z"
)

In [6]:
# scope = ['https://spreadsheets.google.com/feeds']
# credentials = ServiceAccountCredentials.from_json_keyfile_name('jupyter-integration-349314-25735d35924b.json', scope)
# gc = gspread.authorize(credentials)

In [7]:
# spreadsheet_key = "1sCb1YbQ3-1oiL-cnK0yCkwFcQpvpf0efIngAUeC1ixo"
# book = gc.open_by_key(spreadsheet_key)

In [8]:
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 1000

### See if you can query CDC WONDER

### Need:
#### Natality by year
#### Natality by year and race
#### Natality by year and state
#### Grouped or filtered by preventable risk factors
#### Add in violent causes of death

##### attempt to translate variables from icd-10 death code query to natality. did not work

In [10]:
# by-variables" or those parameters selected in the "Group Results By" and the "And By" drop-down lists

b_parameters = {
    "B_1": "D66.V20",  # year
    "B_2": "D66.V42",  # single race
    "B_3": "D66.V43",  # hispanic origin
    "B_4": "*None*",
    "B_5": "*None*",
}

In [11]:
# measures to return, the default measures plus any optional measures

m_parameters = {
    "M_1": "D66.M1",
    # "M_2": "D76.M2",
    # "M_3": "D76.M3",
    # "M_31": "D76.M31",
    # "M_32": "D76.M32",
    # "M_41": "D76.M41",
    # "M_42": "D76.M42"
}

In [12]:
f_parameters = {
    # "F_D66.V1": ["*All*"],  # year/month
    "F_D66.V22": ["*All*"],  # Census Regions - dont change
    #    "F_D66.V2": ["K00-K92"], # ICD-10 Codes
    "F_D66.V37": ["*All*"],  # HHS Regions - dont change
    "F_D66.V21": ["*All*"],  # State  - dont change
}

In [18]:
# variable values to limit in the "where" clause of the query, found in multiple select
# list boxes and advanced finder text entry boxes in the "Request Form."


v_parameters = {
    "V_D66.V21": "*All*",  # States
    "V_D66.V22": "*All*",  # Census Regions
    "V_D66.V37": "*All*",  # HHS Regions
    "V_D66.V154": "*All*",  # 2013 metro/nonmetro
    "V_D66.V155": "*All*",  #  metro/nonmetro
    "V_D66.V156": "*All*",  # 2013 Urbanization
    "V_D66.V157": "*All*",  # 2006 Urbanization
    "V_D66.V1": "*All*",  # age 10
    "V_D66.V38": "*All*",  # age 9
    "V_D66.V39": "*All*",  # age 13
    "V_D66.V40": "*All*",  # age year
    "V_D66.V5": "*All*",  # education
    "V_D66.V27": "*All*",  # marital status
    "V_D66.V43": "*All*",  # hispanic origin
    "V_D66.V4": "*All*",  # expanded hispanic origin
    "V_D66.V43": "*All*",  # hispanic origin
    "V_D66.V42": "*All*",  # single race
    "V_D66.V2": "*All*",  # bridged race
    "V_D66.V20": "*All*",  # Year
    "V_D66.V25": "*All*",  # Month
    "V_D66.V26": "*All*",  # Weekday
    "V_D66.V3": "*All*",  # gender
    "V_D66.V8": "*All*",  # month prenatal care began
    "V_D66.V9": "*All*",  # infant birth weight 12
    "V_D66.V35": "*All*",  # infant birth weight 14
    "V_D66.V41": "*All*",  # infant birth weight 100g
    "V_D66.V28": "*All*",  # birth order
    "V_D66.V7": "*All*",  # plurality/multibirth
    "V_D66.V30": "*All*",  # birthplace
    "V_D66.V31": "*All*",  # delivery method
    "V_D66.V29": "*All*",  # medical attendant
    "V_D66.V6": "*All*",  # gest age 10
    "V_D66.V23": "*All*",  # gest age 11
    "V_D66.V24": "*All*",  # gest age weekly
    "V_D66.V32": "*All*",  # OE gest age 10
    "V_D66.V33": "*All*",  # oe gest age 11
    "V_D66.V34": "*All*",  # oe gest age weekly
    "V_D66.V16": "*All*",  # chronic hypertension
    "V_D66.14": "*All*",  # diabetes
    "V_D66.V17": "*All*",  # pregnancy associated hypertension
    "V_D66.V18": "*All*",  # eclampsia
    "V_D66.V10": "*All*",  # tobacco use
}

In [19]:
# Miscellaneous hidden inputs/parameters usually passed by web form. These do not change.
misc_parameters = {
    "action-Send": "Send",
    "finder-stage-D66.V1": "codeset",
    "finder-stage-D66.V1": "codeset",
    "finder-stage-D66.V2": "codeset",
    "finder-stage-D66.V8": "codeset",
    "finder-stage-D66.V27": "codeset",
    "finder-stage-D66.V9": "codeset",
    "stage": "request",
}

In [20]:
def createParameterList(parameterList):
    """Helper function to create a parameter list from a dictionary object"""

    parameterString = ""

    for key in parameterList:
        parameterString += "<parameter>\n"
        parameterString += "<name>" + key + "</name>\n"

        if isinstance(parameterList[key], list):
            for value in parameterList[key]:
                parameterString += "<value>" + value + "</value>\n"
        else:
            parameterString += "<value>" + parameterList[key] + "</value>\n"

        parameterString += "</parameter>\n"

    return parameterString

In [21]:
xml_request = "<request-parameters>\n"
xml_request += createParameterList(b_parameters)
xml_request += createParameterList(m_parameters)
xml_request += createParameterList(f_parameters)
# xml_request += createParameterList(i_parameters)
# xml_request += createParameterList(o_parameters)
# xml_request += createParameterList(vm_parameters)
# xml_request += createParameterList(v_parameters)
xml_request += createParameterList(misc_parameters)
xml_request += "</request-parameters>"

In [22]:
import requests

url = "https://wonder.cdc.gov/controller/datarequest/D66"
response = requests.post(
    url, data={"request_xml": xml_request, "accept_datause_restrictions": "true"}
)

if response.status_code == 200:
    data = response.text
else:
    print("something went wrong")

#### trying sample with no changes for deaths

In [23]:
# by-variables" or those parameters selected in the "Group Results By" and the "And By" drop-down lists
# in the "Request Form." These "by-variables" are the cross-tabulations, stratifications or indexes
# to the query results. Expect the results data table to show a row for each category in the by-variables,
# and a column for each measure. For example, if you wish to compare data by sex, then "group results by" gender,
# to get a row for females and a row for males in the output.
# M_ are measures to return, the default measures plus any optional measures.

# For this example, will group by year and race

b_parameters = {
    "B_1": "D76.V1-level1",
    "B_2": "D76.V8",
    "B_3": "*None*",
    "B_4": "*None*",
    "B_5": "*None*",
}

In [24]:
# measures to return, the default measures plus any optional measures

# For this example, include deaths, population, and crude rate

m_parameters = {
    "M_1": "D76.M1",  # Deaths, must be included
    "M_2": "D76.M2",  # Population, must be included
    "M_3": "D76.M3",  # Crude rate, must be included
    # "M_31": "D76.M31",        # Standard error (crude rate)
    # "M_32": "D76.M32"         # 95% confidence interval (crude rate)
    "M_41": "D76.M41",  # Standard error (age-adjusted rate)
    "M_42": "D76.M42",  # 95% confidence interval (age-adjusted rate)
}

In [25]:
# values highlighted in a "Finder" control for hierarchical lists,
# such as the "Regions/Divisions/States/Counties hierarchical" list.

# For this example, include all years, months, census regions, hhs regions, states. Only include ICD-10 K00-K92
# for disease of the digestive system

f_parameters = {
    "F_D76.V1": ["*All*"],  # year/month
    "F_D76.V10": ["*All*"],  # Census Regions - dont change
    "F_D76.V2": ["K00-K92"],  # ICD-10 Codes
    "F_D76.V27": ["*All*"],  # HHS Regions - dont change
    "F_D76.V9": ["*All*"],  # State County - dont change
}

In [26]:
# contents of the "Currently selected" information areas next to "Finder" controls in the "Request Form."

# For this example, include all dates, census regions, hhs regions, and states.
# Only include ICD-10 code K00-K92 for disease of the digestive system

i_parameters = {
    "I_D76.V1": "*All* (All Dates)",  # year/month
    "I_D76.V10": "*All* (The United States)",  # Census Regions - dont change
    "I_D76.V2": "K00-K92 (Diseases of the digestive system)",  # ICD-10 Codes
    "I_D76.V27": "*All* (The United States)",  # HHS Regions - dont change
    "I_D76.V9": "*All* (The United States)",  # State County - dont change
}

In [27]:
v_parameters = {
    "V_D76.V1": "",  # Year/Month
    "V_D76.V10": "",  # Census Regions
    "V_D76.V11": "*All*",  # 2006 Urbanization
    "V_D76.V12": "*All*",  # ICD-10 130 Cause List (Infants)
    "V_D76.V17": "*All*",  # Hispanic Origin
    "V_D76.V19": "*All*",  # 2013 Urbanization
    "V_D76.V2": "",  # ICD-10 Codes
    "V_D76.V20": "*All*",  # Autopsy
    "V_D76.V21": "*All*",  # Place of Death
    "V_D76.V22": "*All*",  # Injury Intent
    "V_D76.V23": "*All*",  # Injury Mechanism and All Other Leading Causes
    "V_D76.V24": "*All*",  # Weekday
    "V_D76.V25": "*All*",  # Drug/Alcohol Induced Causes
    "V_D76.V27": "",  # HHS Regions
    "V_D76.V4": "*All*",  # ICD-10 113 Cause List
    "V_D76.V5": ["15-24", "25-34", "35-44"],  # Ten-Year Age Groups
    "V_D76.V51": "*All*",  # Five-Year Age Groups
    "V_D76.V52": "*All*",  # Single-Year Ages
    "V_D76.V6": "00",  # Infant Age Groups
    "V_D76.V7": "*All*",  # Gender
    "V_D76.V8": "*All*",  # Race
    "V_D76.V9": "",  # State/County
}

In [28]:
# other parameters, such as radio buttons, checkboxes, and lists that are not data categories

# For this example, include age-adjusted rates, use ten-year age groups (D76.V5), use state location by default,
# show rates per 100,000, use 2013 urbanization and use ICD-10 Codes (D76.V2) for cause of death category

o_parameters = {
    "O_V10_fmode": "freg",  # Use regular finder and ignore v parameter value
    "O_V1_fmode": "freg",  # Use regular finder and ignore v parameter value
    "O_V27_fmode": "freg",  # Use regular finder and ignore v parameter value
    "O_V2_fmode": "freg",  # Use regular finder and ignore v parameter value
    "O_V9_fmode": "freg",  # Use regular finder and ignore v parameter value
    "O_aar": "aar_std",  # age-adjusted rates
    "O_aar_pop": "0000",  # population selection for age-adjusted rates
    "O_age": "D76.V5",  # select age-group (e.g. ten-year, five-year, single-year, infant groups)
    "O_javascript": "on",  # Set to on by default
    "O_location": "D76.V9",  # select location variable to use (e.g. state/county, census, hhs regions)
    "O_precision": "1",  # decimal places
    "O_rate_per": "100000",  # rates calculated per X persons
    "O_show_totals": "false",  # Show totals for
    "O_timeout": "300",
    "O_title": "Digestive Disease Deaths, by Age Group",  # title for data run
    "O_ucd": "D76.V2",  # select underlying cause of death category
    "O_urban": "D76.V19",  # select urbanization category
}

In [29]:
# values for non-standard age adjusted rates (see mortality online databases).

# For this example, these parameters are ignored as standard age adjusted rates are used

vm_parameters = {
    "VM_D76.M6_D76.V10": "",  # Location
    "VM_D76.M6_D76.V17": "*All*",  # Hispanic-Origin
    "VM_D76.M6_D76.V1_S": "*All*",  # Year
    "VM_D76.M6_D76.V7": "*All*",  # Gender
    "VM_D76.M6_D76.V8": "*All*",  # Race
}

In [30]:
# Miscellaneous hidden inputs/parameters usually passed by web form. These do not change.
misc_parameters = {
    "action-Send": "Send",
    "finder-stage-D76.V1": "codeset",
    "finder-stage-D76.V1": "codeset",
    "finder-stage-D76.V2": "codeset",
    "finder-stage-D76.V27": "codeset",
    "finder-stage-D76.V9": "codeset",
    "stage": "request",
}

In [31]:
def createParameterList(parameterList):
    """Helper function to create a parameter list from a dictionary object"""

    parameterString = ""

    for key in parameterList:
        parameterString += "<parameter>\n"
        parameterString += "<name>" + key + "</name>\n"

        if isinstance(parameterList[key], list):
            for value in parameterList[key]:
                parameterString += "<value>" + value + "</value>\n"
        else:
            parameterString += "<value>" + parameterList[key] + "</value>\n"

        parameterString += "</parameter>\n"

    return parameterString

In [32]:
xml_request = "<request-parameters>\n"
xml_request += createParameterList(b_parameters)
xml_request += createParameterList(m_parameters)
xml_request += createParameterList(f_parameters)
xml_request += createParameterList(i_parameters)
xml_request += createParameterList(o_parameters)
xml_request += createParameterList(vm_parameters)
xml_request += createParameterList(v_parameters)
xml_request += createParameterList(misc_parameters)
xml_request += "</request-parameters>"

In [34]:
import requests

url = "https://wonder.cdc.gov/controller/datarequest/D76"
response = requests.post(
    url, data={"request_xml": xml_request, "accept_datause_restrictions": "true"}
)

if response.status_code == 200:
    data = response.text
else:
    print("something went wrong")

In [35]:
# BeautifulSoup library facilitates parsing of XML response
import bs4 as bs

# This library faciliates 2-dimensional array operations and visualization
import pandas as pd

In [36]:
def xml2df(xml_data):

    root = bs.BeautifulSoup(xml_data, "lxml")
    all_records = []
    row_number = 0
    rows = root.find_all("r")

    for row in rows:
        if row_number >= len(all_records):
            all_records.append([])

        for cell in row.find_all("c"):
            if "v" in cell.attrs:
                try:
                    all_records[row_number].append(
                        float(cell.attrs["v"].replace(",", ""))
                    )
                except ValueError:
                    all_records[row_number].append(cell.attrs["v"])
            else:
                if "r" not in cell.attrs:
                    all_records[row_number].append(cell.attrs["l"])
                else:

                    for row_index in range(int(cell.attrs["r"])):
                        if (row_number + row_index) >= len(all_records):
                            all_records.append([])
                            all_records[row_number + row_index].append(cell.attrs["l"])
                        else:
                            all_records[row_number + row_index].append(cell.attrs["l"])

        row_number += 1
    return all_records

In [37]:
data_frame = xml2df(data)

df = pd.DataFrame(
    data=data_frame,
    columns=[
        "Year",
        "Race",
        "Deaths",
        "Population",
        "Crude Rate",
        "Age-adjusted Rate",
        "Age-adjusted Rate Standard Error",
    ],
)

df.head()

/Users/alexleedsmatthews/.local/share/virtualenvs/notebooks-HSWNtBAS/lib/python3.8/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


,Year,Race,Deaths,Population,Crude Rate,Age-adjusted Rate,Age-adjusted Rate Standard Error
0,1999,American Indian or Alaska Native,210.0,1375207.0,15.3,17.2,1.2
1,1999,Asian or Pacific Islander,73.0,5813970.0,1.3,1.3,0.2
2,1999,Black or African American,1176.0,17026405.0,6.9,7.4,0.2
3,1999,White,5067.0,99715532.0,5.1,5.1,0.1
4,2000,American Indian or Alaska Native,213.0,1438695.0,14.8,16.4,1.1
